# Homework 1

## FINM 36700 - 2024

### UChicago Financial Mathematics

* Mark Hendricks
* hendricks@uchicago.edu

## HBS Case

### *The Harvard Management Company and Inflation-Indexed Bonds*

### Notation
(Hidden LaTeX commands)

$$\newcommand{\mux}{\tilde{\boldsymbol{\mu}}}$$
$$\newcommand{\wtan}{\boldsymbol{\text{w}}^{\text{tan}}}$$
$$\newcommand{\wtarg}{\boldsymbol{\text{w}}^{\text{port}}}$$
$$\newcommand{\mutarg}{\tilde{\boldsymbol{\mu}}^{\text{port}}}$$
$$\newcommand{\wEW}{\boldsymbol{\text{w}}^{\text{EW}}}$$
$$\newcommand{\wRP}{\boldsymbol{\text{w}}^{\text{RP}}}$$
$$\newcommand{\wREG}{\boldsymbol{\text{w}}^{\text{REG}}}$$

***

# 1. HMC's Approach

**Section 1 is not graded**, and you do not need to submit your answers. But you are encouraged to think about them, and we will discuss them.

### 1. 
There are thousands of individual risky assets in which HMC can invest.  Explain why MV optimization across 1,000 securities is infeasible.

### 2.
Rather than optimize across all securities directly, HMC runs a two-stage optimization.
1. They build asset class portfolios with each one optimized over the securities of the specific asset class.  
2. HMC combines the asset-class portfolios into one total optimized portfolio.

In order for the two-stage optimization to be a good approximation of the full MV-optimization on all assets, what must be true of the partition of securities into asset classes?

### 3.
Should TIPS form a new asset class or be grouped into one of the other 11 classes?

### 4. 
Why does HMC focus on real returns when analyzing its portfolio allocation? Is this just a matter of scaling, or does using real returns versus nominal returns potentially change the MV solution?

### 5.
The case discusses the fact that Harvard places bounds on the portfolio allocation rather than implementing whatever numbers come out of the MV optimization problem.

How might we adjust the stated optimization problem in the lecture notes to reflect the extra constraints Harvard is using in their bounded solutions given in Exhibits 5 and 6?

### 6. 
Exhibits 5 shows zero allocation to domestic equities and domestic bonds across the entire computed range of targeted returns, (5.75% to 7.25%). Conceptually, why is the constraint binding in all these cases? What would the unconstrained portfolio want to do with those allocations and why?

### 7.
Exhibit 6 changes the constraints, (tightening them in most cases.) How much deterioration do we see in the mean-variance tradeoff that Harvard achieved?

***

# 2 Mean-Variance Optimization

<i>This section is graded for a good-faith effort by your group. Submit your write-up- along with your supporting code. </i>

### Data
You will need the file in the github repo, `data/multi_asset_etf_data.xlsx`.
- The time-series data gives monthly returns for the 11 asset classes and a short-term Treasury-bill fund return, ("SHV",) which we consider as the risk-free rate.
- The data is provided in total returns, (in which case you should ignore the SHV column,) as well as excess returns, (where SHV has been subtracted from the other columns.)
- These are nominal returns-they are not adjusted for inflation, and in our calculations we are not making any adjustment for inflation.
- The exhibit data that comes via Harvard with the case is unnecessary for our analysis.

### Model
We are going to analyze the problem in terms of **excess** returns.
- Thus, you will focus on the `Excess Returns` section of the lecture notes, especially the formulas on slide 50.
- Be sure to use the`excess returns` tab of the data.

### Format
In the questions below, **annualize the statistics** you report.
- Annualize the mean of monthly returns with a scaling of 12.
- Annualize the volatility of monthly returns with a scaling of $\sqrt{12}$
- The Sharpe Ratio is the mean return divided by the volatility of returns. Accordingly, we can annualize the Sharpe Ratio with a scaling of $\sqrt{12}$
- Note that we are not scaling the raw timeseries data, just the statistics computed from it (mean, vol, Sharpe). 

### Footnotes

#### Data File
* The case does not give time-series data, so this data has been compiled outside of the case, and it intends to represent the main asset classes under consideration via various ETFs. For details on the specific securities/indexes, check the “Info” tab of the data.

#### Risk-free rate
* In the lecture-note we considered a constant risk-free rate. It is okay that our risk-free rate changes over time, but the assumption is that investors know it’s value one-period ahead of time. Thus, at any given point in time, it is a risk-free rate for the next period. (This is often discussed as the "bank account" or "money market account" in other settings.

## 1. Summary Statistics
* Calculate and display the mean and volatility of each asset’s excess return. (Recall we use volatility to refer to standard deviation.)
* Which assets have the best and worst Sharpe ratios? Recall that the Sharpe Ratio is simply the ratio of the mean-to-volatility of excess returns:
$$\text{sharpe ratio of investment }i = \frac{\mux_i}{\sigma_i}$$

## 2. Descriptive Analysis
* Calculate the correlation matrix of the returns. Which pair has the highest correlation? And the lowest?
* How well have TIPS done in our sample? Have they outperformed domestic bonds? Foreign bonds?

## 3. The MV frontier.
* Compute and display the weights of the tangency portfolios: $\wtan$.
* Does the ranking of weights align with the ranking of Sharpe ratios?
* Compute the mean, volatility, and Sharpe ratio for the tangency portfolio corresponding to
$\wtan$.

## 4. TIPS
Assess how much the tangency portfolio (and performance) change if...
* TIPS are dropped completely from the investment set.
* The expected excess return to TIPS is adjusted to be 0.0012 higher than what the historic sample shows.

Based on the analysis, do TIPS seem to expand the investment opportunity set, implying that Harvard should consider them as a separate asset?

In [62]:
import pandas as pd
import numpy as np

excess_returns_df = pd.read_excel(r'/Users/arohansharma/Desktop/assignments/finm-portfolio-2024/data/multi_asset_etf_data.xlsx', 
                                  sheet_name='excess returns', header=0, index_col=0)

# Q.1a Display mean and vol
print('Mean Excess Returns of the Assets')
display(excess_returns_df.mean()*12)
print('Volatility of Excess Returns of the Assets')
display(excess_returns_df.std()*(12**0.5))
# Q.1b Sharpe-Ratio
sharpe_ratios = (excess_returns_df.mean()*12)/(excess_returns_df.std()*(12**0.5))
sharpe_ratios = sharpe_ratios.sort_values()
print('')
print('Asset with highest Sharpe Ratio:', sharpe_ratios.index[-1])
print('Asset with lowest Sharpe Ratio:', sharpe_ratios.index[0])
print('')

# Q.2a Correlation Matrix and Highest and Lowest Correlation
corr_matrix = excess_returns_df.corr()
a = np.matrix((corr_matrix.replace(to_replace=1, value=-np.inf)))
max_corr_idx = np.unravel_index(np.argmax(a, axis=None), a.shape)
print(f'The Asset Pair with Maximum Correlation is {corr_matrix.index[max_corr_idx[0]]}-{corr_matrix.columns[max_corr_idx[1]]}')
a = np.matrix((corr_matrix.replace(to_replace=1, value=np.inf)))
min_corr_idx = np.unravel_index(np.argmin(a, axis=None), a.shape)
print(f'The Asset Pair with Minimum Correlation is {corr_matrix.index[min_corr_idx[0]]}-{corr_matrix.columns[min_corr_idx[1]]}')
print('')
# Q.2b
display(sharpe_ratios[['TIP', 'IEF', 'BWX']])
print('TIPS have outperformed both domestic and foreign bonds on a risk-adjusted basis')
print('')

# Q.3a Find weights of the tangency portfolio
cov_matrix = excess_returns_df.cov()
w_tan = (np.linalg.inv(np.array(cov_matrix)))*((np.matrix(excess_returns_df.mean())).T)
w_tan = w_tan/abs(w_tan.sum())
w_tan = pd.Series(w_tan.T.tolist()[0], index=excess_returns_df.columns)
print('Weights of the Tangency Portfolio:')
display(w_tan.sort_values())
# Q.3b do the weights match the order of Sharpe?
print('Sharpe Ratios:')
display(sharpe_ratios)
print('The weights do not match the order of the Sharpe ratio exactly. This is because the Sharpe ratio takes only the volatility of returns into account while the MV portfolio takes into account the correlation. However, assets with a higher sharpe will tend to have a higher weight')
mean_return = np.matrix(w_tan)*(np.matrix(excess_returns_df.mean())).T
print('')
print(f'Mean Return of Tangency Portfolio: {round(mean_return[0,0]*12*100, 2)}%')
volatility = np.matrix(w_tan)*np.matrix(cov_matrix)*np.matrix(w_tan).T
volatility =(volatility[0,0])**0.5
print(f'Volatility of Tangency Portfolio: {round(volatility*(12**0.5)*100, 2)}%')
sharpe_ratio = (mean_return*12)/(volatility*(12**0.5))
print(f'Sharpe Ratio of the tangency portfolio: {sharpe_ratio[0,0]}')
print('')

# Q.4a Re-assess without TIPS
excess_returns_wo_tips = excess_returns_df[excess_returns_df.columns[:-1]]
cov_matrix_wo_tips = excess_returns_wo_tips.cov()
w_tan_wo_tips = np.linalg.inv(np.array(cov_matrix_wo_tips))*(np.matrix(excess_returns_wo_tips.mean())).T
w_tan_wo_tips = w_tan_wo_tips/abs(w_tan_wo_tips.sum())
w_tan_wo_tips = pd.Series(w_tan_wo_tips.T.tolist()[0], index=excess_returns_wo_tips.columns)
print('Reanalyzing w/o TIPS')
print('Weights of the Tangency Portfolio:')
display(w_tan_wo_tips.sort_values())
print('Sharpe Ratios:')
display(sharpe_ratios)
mean_return = np.matrix(w_tan_wo_tips)*(np.matrix(excess_returns_wo_tips.mean())).T
print('')
print(f'Mean Return of Tangency Portfolio: {round(mean_return[0,0]*12*100, 2)}%')
volatility = np.matrix(w_tan_wo_tips)*np.matrix(cov_matrix_wo_tips)*np.matrix(w_tan_wo_tips).T
volatility =(volatility[0,0])**0.5
print(f'Volatility of Tangency Portfolio: {round(volatility*(12**0.5)*100, 2)}%')
sharpe_ratio = (mean_return*12)/(volatility*(12**0.5))
print(f'Sharpe Ratio of the tangency portfolio: {sharpe_ratio[0,0]}')
print('')

# Q.4b Re-assess with higher expected returns for TIPS
print('Re-analyzing with higher expected returns for TIPS')
cov_matrix = excess_returns_df.cov()
mean_returns = excess_returns_df.mean()
mean_returns.loc['TIP'] += 0.0012/12
w_tan_w_tips_adj = np.linalg.inv(np.array(cov_matrix))*(np.matrix(mean_returns)).T
w_tan_w_tips_adj = w_tan_w_tips_adj/abs(w_tan_w_tips_adj.sum())
w_tan_w_tips_adj = pd.Series(w_tan_w_tips_adj.T.tolist()[0], index=excess_returns_df.columns)
print('Weights of the Tangency Portfolio:')
display(w_tan_w_tips_adj.sort_values())
print('Sharpe Ratios:')
display(sharpe_ratios)
print('')
print(f'Mean Return of Tangency Portfolio: {round(mean_return[0,0]*12*100, 2)}%')
volatility = np.matrix(w_tan_w_tips_adj)*np.matrix(cov_matrix)*np.matrix(w_tan_w_tips_adj).T
volatility =(volatility[0,0])**0.5
print(f'Volatility of Tangency Portfolio: {round(volatility*(12**0.5)*100, 2)}%')
sharpe_ratio = (mean_return*12)/(volatility*(12**0.5))
print(f'Sharpe Ratio of the tangency portfolio: {sharpe_ratio[0,0]}')

Mean Excess Returns of the Assets


BWX   -0.011888
DBC   -0.009086
EEM    0.026960
EFA    0.055037
HYG    0.037356
IEF    0.013939
IYR    0.077912
PSP    0.092851
QAI    0.014959
SPY    0.126983
TIP    0.016844
dtype: float64

Volatility of Excess Returns of the Assets


BWX    0.081671
DBC    0.168455
EEM    0.179940
EFA    0.152203
HYG    0.077289
IEF    0.063197
IYR    0.169585
PSP    0.215238
QAI    0.049007
SPY    0.143066
TIP    0.051258
dtype: float64


Asset with highest Sharpe Ratio: SPY
Asset with lowest Sharpe Ratio: BWX

The Asset Pair with Maximum Correlation is EFA-PSP
The Asset Pair with Minimum Correlation is DBC-IEF



...

The weights do not match the order of the Sharpe ratio. This is because the Sharpe ratio takes only the volatility of returns into account while the MV portfolio takes into account the correlation

Mean Return of Tangency Portfolio: 58.08%
Volatility of Tangency Portfolio: 36.21%
Sharpe Ratio of the tangency portfolio: 1.603881450984104

Re-analyzing with higher expected returns for TIPS
Weights of the Tangency Portfolio:


QAI   -7.819229
BWX   -2.956355
IYR   -0.966587
EFA   -0.538539
PSP   -0.314912
TIP   -0.106419
DBC    0.095195
EEM    0.654112
HYG    0.863494
IEF    4.801132
SPY    5.288108
dtype: float64

Sharpe Ratios:


BWX   -0.145563
DBC   -0.053935
EEM    0.149829
IEF    0.220561
QAI    0.305241
TIP    0.328618
EFA    0.361605
PSP    0.431386
IYR    0.459426
HYG    0.483335
SPY    0.887578
dtype: float64

The weights do not match the order of the Sharpe ratio. This is because the Sharpe ratio takes only the volatility of returns into account while the MV portfolio takes into account the correlation

Mean Return of Tangency Portfolio: 56.96%
Volatility of Tangency Portfolio: 35.51%
Sharpe Ratio of the tangency portfolio: 1.6042840370500056


***

# 3. Allocations

<i>This section is graded for a good-faith effort by your group. Submit your write-up- along with your supporting code.

* Continue with the same data file as the previous section.

* Suppose the investor has a targeted mean excess return (per month) of $\mutarg$ = 0.01.

Build the following portfolios:

#### Equally-weighted (EW)
Rescale the entire weighting vector to have target mean $\mutarg$. Thus, the $i$ element of the weight vector is,
$$\wEW_i = \frac{1}{n}$$

#### “Risk-parity” (RP)
Risk-parity is a term used in a variety of ways, but here we have in mind setting the weight of the portfolio to be proportional to the inverse of its full-sample variance estimate. Thus, the $i$ element of the weight vector is,
$$\wRP_i = \frac{1}{\sigma_i^2}$$

#### Regularized (REG)
Much like the Mean-Variance portfolio, set the weights proportional to 
$$\wREG \sim \widehat{\Sigma}^{-1}\mux$$
but this time, use a regularized covariance matrix,
$$\widehat{\Sigma} = \frac{\Sigma + \Sigma_D}{2}$$
where $\Sigma_D$ denotes a *diagonal* matrix of the security variances, with zeros in the off-diagonals.

Thus, $\widehat{\Sigma}$ is obtained from the usual covariance matrix, $\Sigma$, but shrinking all the covariances to half their estimated values. 


### Comparing

In order to compare all these allocation methods, (those above, along with the tangency portfolio obtained in the previous section,) rescale each weight vector, such that it has targeted mean return of $\mutarg$.

* Calculate the performance of each of these portfolios over the sample.
* Report their mean, volatility, and Sharpe ratio. 
* How do these compare across the four allocation methods?

In [79]:
def summary_statistics_annualized(returns, annual_factor = 12):
    """This functions returns the summary statistics for the input total/excess returns passed
    into the function"""
    
    summary_statistics = pd.DataFrame(index=returns.columns)
    summary_statistics['Mean'] = returns.mean() * annual_factor
    summary_statistics['Vol'] = returns.std() * np.sqrt(annual_factor)
    summary_statistics['Sharpe'] = (returns.mean() / returns.std()) * np.sqrt(annual_factor)
    
    return summary_statistics

target_mean = 0.01

w_eq = [1/len(excess_returns_df.columns)]*len(excess_returns_df.columns)
w_rp = list(1/np.array(excess_returns_df.var()))
cov_reg = (excess_returns_df.cov()/2 + np.diag(excess_returns_df.var()))/2
w_reg = np.linalg.inv(cov_reg)*(np.matrix(excess_returns_df.mean())).T

weights_df = pd.DataFrame(columns=['tangency', 'equal', 'risk_parity', 'regularized'])
weights_df['tangency'] = w_tan
weights_df['equal'] = w_eq
weights_df['risk_parity'] = w_rp
weights_df['regularized'] = w_reg
weights_df.index = excess_returns_df.columns
weights_df.fillna(0, inplace=True)
weights_df

weights_df *= target_mean / (excess_returns_df.mean()@weights_df)

display(summary_statistics_annualized(excess_returns_df@weights_df))

print("Amongst the four allocations, the tangency portfolio has the highest Sharpe and lowest Vol (as expected) followed by the regularized weighting scheme. The risk parity weighted portfolio has the highest vol and the lowest Sharpe")

,Mean,Vol,Sharpe
tangency,0.12,0.074765,1.605037
equal,0.12,0.291527,0.411625
risk_parity,0.12,0.309515,0.387703
regularized,0.12,0.118010,1.016864


Amongst the four allocations, the tangency portfolio has the highest Sharpe and lowest Vol (as expected) followed by the regularized weighting scheme. The risk parity weighted portfolio has the highest vol and the lowest Sharpe


***

# 4. Out-of-Sample Performance

<i>This section is not graded, and you do not need to submit it. Still, we may discuss it in class, in which case, you would be expected to know it.

## 1. One-step Out-of-Sample (OOS) Performance
Let’s divide the sample to both compute a portfolio and then check its performance out of sample.
* Using only data through the end of `2022`, compute the weights built in Section 3.
* Rescale the weights, (using just the in-sample data,) to set each allocation to have the same mean return of $\mutarg$.
* Using those weights, calculate the portfolio’s Sharpe ratio within that sample.
* Again using those weights, (derived using data through `2022`,) calculate the portfolio’s OOS Sharpe ratio, which is based only on performance in `2023-2024`.

## 2. Rolling OOS Performance

Iterate the Out-of-Sample performance every year, not just the final year. Namely,
* Start at the end of `2015`, and calculate the weights through that time. Rescale them using the mean returns through that time.
* Apply the weights to the returns in the upcoming year, (`2016`.)
* Step forward a year in time, and recompute.
* Continue until again calculating the weights through `2023` and applying them to the returns in `2024`.

Report the mean, volatility, and Sharpe from this dynamic approach for the following portfolios:
* mean-variance (tangency)
* equally-weighted
* risk-parity
* regularized

***

# 5. Without a Riskless Asset

<i>This section is not graded, and you do not need to submit it. Still, we may discuss it in class, in which case, you would be expected to know it.

Re-do Section 2 above, but in the model without a risk-free rate.

That is, build the MV allocation using the two-part formula in the `Mean-Variance` section of the notes.
* This essentially substitutes the risk-free rate with the minimum-variance portfolio.
* Now, the allocation depends nonlinearly on the target mean return, $\mutarg$. (With a risk-free rate, we simply scale the weights up and down to achieve the mean return.)

You will find that, conceptually, the answers are very similar. 

***